# Random investigations into the AoC data
When I have time to look into it, I will...

Set up is below. 

[Jump to create dataframe](#Create-dataframe)

In [1]:
import pandas as pd
import requests

There's a stats page for each year of AoC at `adventofcode.com/YYYY/stats`.

Can I scrape the info from the page to do some analysis?

Note: the first year of Advent of Code was 2015.

I'm holding off on 2023 until later...I'm too sleepy.

In [2]:
from bs4 import BeautifulSoup as bs

In [3]:
def get_request(year):
    url = f'https://adventofcode.com/{year}/stats'
    return requests.get(url)

In [4]:
def get_spans_from_soup(soup):
    '''
    Parameters
    ----------
    soup: beautiful soup object from the content of a request
    
    Returns
    -------
    list of texts from span tags in the soup
    '''
    return [span.get_text() for span in soup.find_all('span')]

In [5]:
from collections import namedtuple
Star = namedtuple('Star','year day gold silver')

In [6]:
def get_star_counts(spans):
    '''
    Parameters
    ----------
    spans: list of text values of soup spans
    
    Returns
    -------
    list of Stars (star data for the year)
    '''
    #  the starting index of spans and the mod values are very specific to the format
    values = [int(value.strip()) for i,value in enumerate(spans[6:]) if (i%4==0)|(i%4==1)]
    stars = [Star(year, 25-i//2, values[i],values[i+1]) for i in range(0,len(values),2)]
    return stars

Put this all together...

In [7]:
def get_data_for_year(year):
    r = get_request(year)
    soup = bs(r.content)
    spans = get_spans_from_soup(soup)
    stars = get_star_counts(spans)
    return pd.DataFrame(stars)
    

In [8]:
data = list()
year = 2022
while year>=2015:
    data.append(get_data_for_year(year))
    year-=1
len(data)

8

In [9]:
df = pd.concat(data, ignore_index=True)
df.shape

(200, 4)

[Return to top](#Random-investigations-into-the-AoC-data)

## Create dataframe 

Run all above selected cell to get the data into dataframe `df`.

In [10]:
df.describe()[['gold','silver']]

,gold,silver
count,200.00000,200.000000
mean,37225.33000,3349.380000
std,46274.32049,4918.661594
min,3933.00000,24.000000
25%,9811.75000,474.000000
50%,18443.50000,1452.000000
75%,46571.25000,4623.250000
max,279022.00000,39679.000000


In [12]:
df['percent_gold'] = df['gold']/(df['gold']+df['silver'])
df

,year,day,gold,silver,percent_gold
0,2022,25,12978,7299,0.640036
1,2022,24,16162,242,0.985248
2,2022,23,18310,249,0.986583
3,2022,22,15176,5079,0.749247
4,2022,21,22799,3033,0.882587
...,...,...,...,...,...
195,2015,5,36212,6289,0.852027
196,2015,4,41976,1453,0.966543
197,2015,3,48548,3971,0.924389
198,2015,2,58430,5942,0.907693


[Return to top](#Random-investigations-into-the-AoC-data)